# Machine Learning Engineer Nanodegree
## Supervised Learning
## Project: Building a Student Intervention System

Welcome to the second project of the Machine Learning Engineer Nanodegree! In this notebook, some template code has already been provided for you, and it will be your job to implement the additional functionality necessary to successfully complete this project. Sections that begin with **'Implementation'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section and the specifics of the implementation are marked in the code block with a `'TODO'` statement. Please be sure to read the instructions carefully!

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.  

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

### Question 1 - Classification vs. Regression
*Your goal for this project is to identify students who might need early intervention before they fail to graduate. Which type of supervised learning problem is this, classification or regression? Why?*

**Answer: **
This is binary classification type of supervised learning problem, because all data set has labeled (in feature 'passed') with just one of the two possible discrete values ('yes' or 'no').

## Exploring the Data
Run the code cell below to load necessary Python libraries and load the student data. Note that the last column from this dataset, `'passed'`, will be our target label (whether the student graduated or didn't graduate). All other columns are features about each student.

In [1]:
# Import libraries
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score

# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"

Student data read successfully!


### Implementation: Data Exploration
Let's begin by investigating the dataset to determine how many students we have information on, and learn about the graduation rate among these students. In the code cell below, you will need to compute the following:
- The total number of students, `n_students`.
- The total number of features for each student, `n_features`.
- The number of those students who passed, `n_passed`.
- The number of those students who failed, `n_failed`.
- The graduation rate of the class, `grad_rate`, in percent (%).


In [2]:
# TODO: Calculate number of students
n_students = len(student_data)

# TODO: Calculate number of features
n_features = len(student_data.columns[:-1])

# TODO: Calculate passing students
n_passed = len(student_data[student_data['passed']=='yes'])

# TODO: Calculate failing students
n_failed = len(student_data[student_data['passed']=='no'])

# TODO: Calculate graduation rate
grad_rate = 100.0 * n_passed / (n_passed + n_failed)

# Print the results
print "Total number of students: {}".format(n_students)
print "Number of features: {}".format(n_features)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of features: 30
Number of students who passed: 265
Number of students who failed: 130
Graduation rate of the class: 67.09%


## Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Run the code cell below to separate the student data into feature and target columns to see if any features are non-numeric.

In [3]:
# Extract feature columns
feature_cols = list(student_data.columns[:-1])

# Extract target column 'passed'
target_col = student_data.columns[-1] 

# Show the list of columns
print "Feature columns:\n{}".format(feature_cols)
print "\nTarget column: {}".format(target_col)

# Separate the data into feature data and target data (X_all and y_all, respectively)
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# Show the feature information by printing the first five rows
print "\nFeature values:"
print X_all.head()

Feature columns:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...       

### Preprocess Feature Columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation. Run the code cell below to perform the preprocessing routine discussed in this section.

In [4]:
def preprocess_features(X):
    ''' Preprocesses the student data and converts non-numeric binary variables into
        binary (0/1) variables. Converts categorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():
        
        # If data type is non-numeric, replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print "Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Implementation: Training and Testing Data Split
So far, we have converted all _categorical_ features into numeric values. For the next step, we split the data (both features and corresponding labels) into training and test sets. In the following code cell below, you will need to implement the following:
- Randomly shuffle and split the data (`X_all`, `y_all`) into training and testing subsets.
  - Use 300 training points (approximately 75%) and 95 testing points (approximately 25%).
  - Set a `random_state` for the function(s) you use, if provided.
  - Store the results in `X_train`, `X_test`, `y_train`, and `y_test`.

In [5]:
# TODO: Import any additional functionality you may need here
from sklearn.model_selection import train_test_split

# TODO: Set the number of training points
num_train = 300

# Set the number of testing points
num_test = X_all.shape[0] - num_train

# TODO: Shuffle and split the dataset into the number of training and testing points above
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=num_test, random_state=101)


# Show the results of the split
print "Training set has {} samples.".format(X_train.shape[0])
print "Testing set has {} samples.".format(X_test.shape[0])

Training set has 300 samples.
Testing set has 95 samples.


## Training and Evaluating Models
In this section, you will choose 3 supervised learning models that are appropriate for this problem and available in `scikit-learn`. You will first discuss the reasoning behind choosing these three models by considering what you know about the data and each model's strengths and weaknesses. You will then fit the model to varying sizes of training data (100 data points, 200 data points, and 300 data points) and measure the F<sub>1</sub> score. You will need to produce three tables (one for each model) that shows the training set size, training time, prediction time, F<sub>1</sub> score on the training set, and F<sub>1</sub> score on the testing set.

**The following supervised learning models are currently available in** [`scikit-learn`](http://scikit-learn.org/stable/supervised_learning.html) **that you may choose from:**
- Gaussian Naive Bayes (GaussianNB)
- Decision Trees
- Ensemble Methods (Bagging, AdaBoost, Random Forest, Gradient Boosting)
- K-Nearest Neighbors (KNeighbors)
- Stochastic Gradient Descent (SGDC)
- Support Vector Machines (SVM)
- Logistic Regression

### Question 2 - Model Application
*List three supervised learning models that are appropriate for this problem. For each model chosen*
- Describe one real-world application in industry where the model can be applied. *(You may need to do a small bit of research for this — give references!)* 
- What are the strengths of the model; when does it perform well? 
- What are the weaknesses of the model; when does it perform poorly?
- What makes this model a good candidate for the problem, given what you know about the data?

**Answer: **
#### Least but not last: Decision Tree Classifier, AdaBoost Classifier (with Decision Tree as base estimator), SVM are appropriate models for this
- Decision Tree Classifiers:
    + Real-world application:
        + Decision Tree methods are one of the most popular of inductive inference algorithms.
        + They have been applied to a broad range of tasks from medical diagnose to credit or risk analysis.
        + Decision Tree Classifier has often been used in medicine and medical research and engineering, one example:
        + P. KOKOL, M. MERNIK, J. ZAVRSNIK, AND K. KANCLER. Decision trees based on automatic learning and their use in cardiology. Journal of Medical Systems, 18(4):201, 1994. (http://link.springer.com/article/10.1007/BF00996704)
    + Strength:
        + Relatively easy to understand and implement due to its simplicity
        + Handle high dimentional feature set well in terms of complexity
        + No need to normalize or scale features
    + Weakness:
        + Prune to overfit, especially with large number of features and large trees, high dimentional feature space requires lots of data
        + Training time depend heavily upon an update of training set (limitation to tree based algorithm, prune to re-draw)
        + Absolutely ignore relationship among feature variables (it consoder all features be orthogonal)
    + Applicability (to this problem):
        + Most features are already binary or catgorical and independent, natively suited as decision variables.

- AdaBoost Classifier:
    + Real-world application:
        + Boosting is a general strategy to improve accuracy of a learning algorithm.
        + It combines multiple weak classifiers, which perform just slightly better than random guess, to form a stronger learner.
        + As a boosting method, Adaboost has been successfully used in computer vision such as Face detection techniques, for example:
        + Robust Real-Time Face Detection, PAUL VIOLA, Microsoft Reasearch (https://www.vision.caltech.edu/html-files/EE148-2005-Spring/pprs/viola04ijcv.pdf)
    + Strength:
        + As an ensemble method, it is fexible in choosing base estimator
        + Start with weak learner then focus on boosting is an easier and practical approach than just to find a strong learner with great performance, rule of thumb for weak learner is relatively easier to identify
        + Fast and no many parameters to tune
        + Resistant to overfitting
    + Weakness:
        + Time to fit/train, number of estimators to boost is normally not small, high iteration cost of time
        + Sensitive to noise and outliers, as those will be refocused, so proper moderation is needed to prevent this 
    + Applicability (to this problem):
        + Most features already binary or catgorical and independent, Adaboost with dumb decision tree as weak learner suited well.
        + With ~40 features but just 300 training data, Adaboost is helpful to prevent overfit
        
- Support Vector Machines:
    + Real-world application:
        + SVM is a relatively new technique suited for binary or multiclass classifications. It works by maximum inter-class margins.
        + SVM has empirically good performance with successful applications in numerous fields (bioinformatics, text, hand-writing recognition, pattern recognition, etc.).
        + Boser, Guyon & Vapnik explored the First real-world application of SVMs - Handwritten Digital Recognition:
        + A Training Algorithm for Optimal Margin Classiers (http://w.svms.org/training/BOGV92.pdf)
        + Support-Vector Networks (http://image.diku.dk/imagecanon/material/cortes_vapnik95.pdf)
    + Strength:
        + It provides good generalization, when parameters C and gamma (if Gaussian kernel) are appropriately chosen. 
        + It handles none-linear separable data via kernel trick, kernel functions can transform input data to higher dimentional feature space with data becoming more separable.
        + Different stock kernels available, and kernel is even customizable
    + Weakness:
        + Features are very different in scale may need to be normalized for better accurarcy
        + Some slow with large data set and large number of features
        + Prune to noise and none clear margin in data
    + Applicability (to this problem):
        + Neither data set size nor feature count are big enough to concern
        + It is a classic problem of binary classification

### Setup
Run the code cell below to initialize three helper functions which you can use for training and testing the three supervised learning models you've chosen above. The functions are as follows:
- `train_classifier` - takes as input a classifier and training data and fits the classifier to the data.
- `predict_labels` - takes as input a fit classifier, features, and a target labeling and makes predictions using the F<sub>1</sub> score.
- `train_predict` - takes as input a classifier, and the training and testing data, and performs `train_clasifier` and `predict_labels`.
 - This function will report the F<sub>1</sub> score for both the training and testing data separately.

In [6]:
def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print "Trained model in {:.4f} seconds".format(end - start)

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Print and return results
    print "Made predictions in {:.4f} seconds.".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print "Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    print "F1 score for training set: {:.4f}.".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {:.4f}.".format(predict_labels(clf, X_test, y_test))

### Implementation: Model Performance Metrics
With the predefined functions above, you will now import the three supervised learning models of your choice and run the `train_predict` function for each one. Remember that you will need to train and predict on each classifier for three different training set sizes: 100, 200, and 300. Hence, you should expect to have 9 different outputs below — 3 for each model using the varying training set sizes. In the following code cell, you will need to implement the following:
- Import the three supervised learning models you've discussed in the previous section.
- Initialize the three models and store them in `clf_A`, `clf_B`, and `clf_C`.
 - Use a `random_state` for each model you use, if provided.
 - **Note:** Use the default settings for each model — you will tune one specific model in a later section.
- Create the different training set sizes to be used to train each model.
 - *Do not reshuffle and resplit the data! The new training points should be drawn from `X_train` and `y_train`.*
- Fit each model with each training set size and make predictions on the test set (9 in total).  
**Note:** Three tables are provided after the following code cell which can be used to store your results.

In [7]:
# TODO: Import the three supervised learning models from sklearn
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier

# TODO: Initialize the three models
# Here I have found my three appropriate classifiers to begin with
clf_A = DecisionTreeClassifier(criterion='gini', max_depth=6, random_state = 10)
clf_B = AdaBoostClassifier(n_estimators=200, learning_rate=0.2, random_state = 11)
clf_C = SVC(C=1.0, kernel='rbf', gamma='auto', random_state = 12)

# TODO: Set up the training set sizes
X_train_100 = X_train.iloc[0:100]
y_train_100 = y_train.iloc[0:100]

X_train_200 = X_train.iloc[0:200]
y_train_200 = y_train.iloc[0:200]

X_train_300 = X_train.iloc[0:300]
y_train_300 = y_train.iloc[0:300]

# TODO: Execute the 'train_predict' function for each classifier and each training set size
train_predict(clf_A, X_train_100, y_train_100, X_test, y_test)
train_predict(clf_A, X_train_200, y_train_200, X_test, y_test)
train_predict(clf_A, X_train_300, y_train_300, X_test, y_test)
print ""
train_predict(clf_B, X_train_100, y_train_100, X_test, y_test)
train_predict(clf_B, X_train_200, y_train_200, X_test, y_test)
train_predict(clf_B, X_train_300, y_train_300, X_test, y_test)
print ""
train_predict(clf_C, X_train_100, y_train_100, X_test, y_test)
train_predict(clf_C, X_train_200, y_train_200, X_test, y_test)
train_predict(clf_C, X_train_300, y_train_300, X_test, y_test)
print ""

# Also report default performance of 3 selected models below w/. refactored code :)
# Will see default performance is some lower than above none extensive optimized 1s 
print "----- Performance of 3 default models without any tuning -----"
for clf in [DecisionTreeClassifier(random_state = 10), AdaBoostClassifier(random_state = 11), SVC(random_state = 12)]:
    
    # show classifier type
    print "\n{}: \n".format(clf.__class__.__name__)
    
    # loop through different training size (i.e. 100, 200, 300)
    for n in [100, 200, 300]:
        train_predict(clf, X_train[:n], y_train[:n], X_test, y_test)

print ""
        
# Also we know that majority (67.09%) students will graduate given this dataset.
# Let's find out the f1_score of all 'yes' dummy predictor, just as a benchmark.
# In the end we should produce a model way better than the dummy!
print "----- Performance reference from all 'yes' dummy predict -----"

print "\nF1 score for predicting all 'yes': {:.4f}.".format(
    f1_score(y_true = ['yes']*n_passed + ['no']*n_failed, y_pred = ['yes']*n_students, pos_label='yes'))

print "\nFinal model tuned should perform better !"

Training a DecisionTreeClassifier using a training set size of 100. . .
Trained model in 0.0011 seconds
Made predictions in 0.0003 seconds.
F1 score for training set: 0.9605.
Made predictions in 0.0002 seconds.
F1 score for test set: 0.7023.
Training a DecisionTreeClassifier using a training set size of 200. . .
Trained model in 0.0010 seconds
Made predictions in 0.0003 seconds.
F1 score for training set: 0.9524.
Made predictions in 0.0002 seconds.
F1 score for test set: 0.7200.
Training a DecisionTreeClassifier using a training set size of 300. . .
Trained model in 0.0012 seconds
Made predictions in 0.0002 seconds.
F1 score for training set: 0.9052.
Made predictions in 0.0001 seconds.
F1 score for test set: 0.7939.

Training a AdaBoostClassifier using a training set size of 100. . .
Trained model in 0.4462 seconds
Made predictions in 0.0139 seconds.
F1 score for training set: 0.9669.
Made predictions in 0.0138 seconds.
F1 score for test set: 0.7463.
Training a AdaBoostClassifier using

### Tabular Results
Edit the cell below to see how a table can be designed in [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). You can record your results from above in the tables provided.

##### N.B. Data recorded in following tables are from the 1st manually played around models vs. default ones (reported above also)

** Classifer 1 - DecisionTreeClassifier**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |         0.0011          |         0.0002         |      0.9605      |      0.7023     |
| 200               |         0.0010          |         0.0002         |      0.9524      |      0.7200     |
| 300               |         0.0012          |         0.0001         |      0.9052      |      0.7939     |

** Classifer 2 - AdaBoostClassifier (200 rounds of DecisionTree as base estimator)**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |         0.4462          |         0.0138         |      0.9669      |      0.7463     |
| 200               |         0.4153          |         0.0142         |      0.8903      |      0.8028     |
| 300               |         0.4572          |         0.0133         |      0.8598      |      0.8029     |

** Classifer 3 - SVM**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |         0.0011          |         0.0006         |      0.8639      |      0.7975     |
| 200               |         0.0027          |         0.0016         |      0.8677      |      0.8077     |
| 300               |         0.0077          |         0.0017         |      0.8621      |      0.8077     |

## Choosing the Best Model
In this final section, you will choose from the three supervised learning models the *best* model to use on the student data. You will then perform a grid search optimization for the model over the entire training set (`X_train` and `y_train`) by tuning at least one parameter to improve upon the untuned model's F<sub>1</sub> score. 

### Question 3 - Choosing the Best Model
*Based on the experiments you performed earlier, in one to two paragraphs, explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?*

**Answer: **
#### From experiments above, I choose SVM classifier as the best supervisor learning model for this problem to further exploration, because:
- SVM provides the smallest performance gap between training score and test score, so it seems general enough, and it is not underperform in both training and predict time given the dataset
- AdaBoost is better than DecisionTree in not overfit, but it doesn't show great performance gain at higher cost of time (many rounds of base estimators)
- DecisionTree is the one most overfitted here

### Question 4 - Model in Layman's Terms
*In one to two paragraphs, explain to the board of directors in layman's terms how the final model chosen is supposed to work. Be sure that you are describing the major qualities of the model, such as how the model is trained and how the model makes a prediction. Avoid using advanced mathematical or technical jargon, such as describing equations or discussing the algorithm implementation.*

**Answer: **
#### The final model is chosen to be a Support Vector Machine Classifier, it works to yield the maximum margin to separate `passed` and `failed` students by constructing hyperplane using the provided multiple features (e.g. `age`, `sex`, `family size`, `family support`, `study time`, ...) to each student. A couple of hyperparameters exist to the model, we use exhaustic search called Grid cross validation to find best estimator with these parameters be tuned:
- `kernel` type: function type to approximate the distance between two samples in their featute space
- penalty of misclassification `C`: larger value means a harder margin to ask, and a higher penalty for each misclassified data point
- influence of a data point on deciding class boundries `gamma`: to non-linear kernels large `gamma` means far away data points have less influence to the class boundry, versus closer points have more power in deciding boundries, this leads to high variance model; smaller `gamma` implies that a data point will have more influence on the class boundry even its distance to boundry is some large.

#### The model is trained below by iterating though each parameters combination in the parameters dictionary, for three types kernels respectively. For each parameter combine group (i.e. a determined `kernel`, `C`, `gamma`, etc.) we run cross validation of default 3-fold (StratifiedKFold) on training data set (300 samples) to find the performance of chosen parameters, final best estimator is chosen among all the estimators for the best f1 score.

#### To non-linear general kernels (e.g. RBF), it is difficult to interpret the SVM feature weights or importance or scores, since kernel trick has transformed feature space. But to make an intuitive explaination to none technical audience, below is how the model select works in Layman's Terms:



The SVM takes data about previous students (age, gender, family, etc), and uses them to create a function that draws a boundary between the students who passed and those who didn't. The boundary should be drawn so as to maximize the distance between the group of students who passed and those didn't.

<b>Conceptual Figure 1. (from Udacity repo)</b>
+ Blue circles represent 'Passed' students
+ Red squares represent 'Failed' students
+ X<sub>1</sub> represents a composite feature built from input features (age, gender, family, etc)
+ X<sub>2</sub> represents another composite feature from input features (study time, free time, health, etc)
+ the model works by finding the best composite features (e.g. X<sub>1</sub>, X<sub>2</sub>, etc.) giving maximum margin

!['Figure 1'](optimal-hyperplane.png)

Often, though, it's not easy to draw a decision boundary in low dimensions of original input space, so the SVM model often separates the passing and failing students by constructing higher dimentional space with composited features, with a hyperplane in separating the hyperspace easyier to find.

<b>Conceptual Figure 2. (from Udacity repo)</b>
+ Blue points represent 'Passed' students
+ Red points represent 'Failed' students
+ Left represents a very curvly (mathematic non-feasible) classification boundry on 2-D plane from two original input features (e.g. age, family support)
+ Right side represents SVM hyperplane ideally separate `Passed` vs. `Failed` students in a virtual 3-D space from 3 composite features found
+ The transform between Left and Right represents SVM kernel trick and inner method to find higher dimentions to separate samples 


!['Figure 2'](1gvce.png)


Using this model (hyperspace and hyperplanes) created with existing students data that we know passed or not, the SVM model can predict a student `pass` or `fail` by looking at the student's data in higher dimentional hyperspace.

### Implementation: Model Tuning
Fine tune the chosen model. Use grid search (`GridSearchCV`) with at least one important parameter tuned with at least 3 different values. You will need to use the entire training set for this. In the code cell below, you will need to implement the following:
- Import [`sklearn.grid_search.GridSearchCV`](http://scikit-learn.org/0.17/modules/generated/sklearn.grid_search.GridSearchCV.html) and [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Create a dictionary of parameters you wish to tune for the chosen model.
 - Example: `parameters = {'parameter' : [list of values]}`.
- Initialize the classifier you've chosen and store it in `clf`.
- Create the F<sub>1</sub> scoring function using `make_scorer` and store it in `f1_scorer`.
 - Set the `pos_label` parameter to the correct value!
- Perform grid search on the classifier `clf` using `f1_scorer` as the scoring method, and store it in `grid_obj`.
- Fit the grid search object to the training data (`X_train`, `y_train`), and store it in `grid_obj`.

In [8]:
# TODO: Import 'GridSearchCV' and 'make_scorer'
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer

# TODO: Create the parameters list you wish to tune
parameters = [
    { 'kernel': ['linear'], 'C': [1, 5, 10, 20, 50, 100] },
    { 'kernel': ['poly'], 'C': [1, 5, 10, 20, 50, 100], 'gamma': [0.001, 0.01, 0.1, 1.0, 10, 100], 'degree': [2, 3, 4, 5] },
    { 'kernel': ['rbf'], 'C': [1, 5, 10, 20, 50, 100], 'gamma': [0.001, 0.002, 0.005, 0.01, 0.02, 0.05, 0.1, 1.0, 10, 100]}
 ]

# TODO: Initialize the classifier
clf = SVC()

# TODO: Make an f1 scoring function using 'make_scorer' 
f1_scorer = make_scorer(f1_score, pos_label='yes')

# TODO: Perform grid search on the classifier using the f1_scorer as the scoring method
# Use the default 3-fold (StratifiedKFold since estimator is binary/multiclass classifier) cross validation
grid_obj = GridSearchCV(estimator = clf, param_grid = parameters, scoring = f1_scorer)

# TODO: Fit the grid search object to the training data and find the optimal parameters
grid_obj = grid_obj.fit(X_train, y_train)

# Get the estimator
clf = grid_obj.best_estimator_

# Report the final F1 score for training and testing after parameter tuning
print "Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train))
print "Tuned model has a testing F1 score of {:.4f}.".format(predict_labels(clf, X_test, y_test))

Made predictions in 0.0066 seconds.
Tuned model has a training F1 score of 0.9157.
Made predictions in 0.0031 seconds.
Tuned model has a testing F1 score of 0.8182.


### Question 5 - Final F<sub>1</sub> Score
*What is the final model's F<sub>1</sub> score for training and testing? How does that score compare to the untuned model?*

**Answer: **
#### the final model's F<sub>1</sub> scores are:
  Training F<sub>1</sub>: 0.9157
  
  Testing  F<sub>1</sub>: 0.8182

#### Compare to untuned model, it:
+ Yields better training score from hyperparameters search and cross-validation (N.B. this final training score may be incomparable to previous untuned number, because CV never used 100% of 300 samples for training, it also picked mean score from k folds as score, finally k is a parameter that I did not tune here, just used default - 3)
+ Also generates higher testing score due to a higher bias/viance balance from 3-fold CV with better data utilization for generalization



## Additional tips

### Feature Scaling

We know that not all models require feature scaling (e.g. DecisionTree not need), but SVM is one of models that benefits from feature scaling (because it calculates spatial distance by multiple features). All work above did not implement feature scaling, which is probably fine given the difference in scale among original features is small in this problem. 

But still, let give it a try.

In [10]:
from sklearn.preprocessing import Normalizer
normer = Normalizer()
X_train = normer.fit_transform(X_train)
X_test = normer.transform(X_test)

C_range = np.logspace(-2, 2, 13)
gamma_range = np.logspace(-3, 1, 13)
parameters = dict(gamma=gamma_range, C=C_range)

# The default grid search uses 3 folds; use the 'cv' param to change this
grid_obj = GridSearchCV(clf, parameters, cv=4,
                        scoring=f1_scorer, verbose=1,
                        n_jobs=-1, pre_dispatch='2*n_jobs')

grid_obj = grid_obj.fit(X_train, y_train)

# Get the estimator
clf = grid_obj.best_estimator_

# Report the final F1 score for training and testing after parameter tuning
print "Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train))
print "Tuned model has a testing F1 score of {:.4f}.".format(predict_labels(clf, X_test, y_test))

Fitting 4 folds for each of 169 candidates, totalling 676 fits


[Parallel(n_jobs=-1)]: Done 368 tasks      | elapsed:    1.2s
[Parallel(n_jobs=-1)]: Done 676 out of 676 | elapsed:    1.6s finished


Made predictions in 0.0033 seconds.
Tuned model has a training F1 score of 0.8465.
Made predictions in 0.0014 seconds.
Tuned model has a testing F1 score of 0.8511.


### As we see from the testing F<sub>1</sub> score - `SVM produces better performance matrix with feature scaling` 

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  
**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.